In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from sklearn.model_selection import KFold
from tensorflow import keras

In [2]:
train = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")
test = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
sample_submission = pd.read_csv("../input/petfinder-pawpularity-score/sample_submission.csv")

train["file_path"] = train["Id"].apply(lambda x: "../input/petfinder-pawpularity-score/train/" + x + ".jpg")
test["file_path"] = test["Id"].apply(lambda x: "../input/petfinder-pawpularity-score/test/" + x + ".jpg")

In [3]:
tabular_columns = ['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']
image_size = 224
batch_size = 64
epochs = 20
train_on_fold = None # Which Fold to train, train on all folds if None
AUTO = tf.data.experimental.AUTOTUNE

In [4]:
def random_erasing(img, sl=0.1, sh=0.2, rl=0.4, p=0.3):
    h = tf.shape(img)[0]
    w = tf.shape(img)[1]
    c = tf.shape(img)[2]
    origin_area = tf.cast(h*w, tf.float32)

    e_size_l = tf.cast(tf.round(tf.sqrt(origin_area * sl * rl)), tf.int32)
    e_size_h = tf.cast(tf.round(tf.sqrt(origin_area * sh / rl)), tf.int32)

    e_height_h = tf.minimum(e_size_h, h)
    e_width_h = tf.minimum(e_size_h, w)

    erase_height = tf.random.uniform(shape=[], minval=e_size_l, maxval=e_height_h, dtype=tf.int32)
    erase_width = tf.random.uniform(shape=[], minval=e_size_l, maxval=e_width_h, dtype=tf.int32)

    erase_area = tf.zeros(shape=[erase_height, erase_width, c])
    erase_area = tf.cast(erase_area, tf.uint8)

    pad_h = h - erase_height
    pad_top = tf.random.uniform(shape=[], minval=0, maxval=pad_h, dtype=tf.int32)
    pad_bottom = pad_h - pad_top

    pad_w = w - erase_width
    pad_left = tf.random.uniform(shape=[], minval=0, maxval=pad_w, dtype=tf.int32)
    pad_right = pad_w - pad_left

    erase_mask = tf.pad([erase_area], [[0,0],[pad_top, pad_bottom], [pad_left, pad_right], [0,0]], constant_values=1)
    erase_mask = tf.squeeze(erase_mask, axis=0)
    erased_img = tf.multiply(tf.cast(img,tf.float32), tf.cast(erase_mask, tf.float32))

    return tf.cond(tf.random.uniform([], 0, 1) > p, lambda: tf.cast(img, img.dtype), lambda:  tf.cast(erased_img, img.dtype))

In [5]:
def data_augment(image):
    image = tf.image.random_flip_left_right(image)
    image = random_erasing(image)
    return image

In [6]:
def preprocess_training(image_url, tabular):
    image_string = tf.io.read_file(image_url)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = data_augment(image)
    image = tf.image.central_crop(image, 1.0)
    image = tf.image.resize(image, (image_size, image_size))
    return (image, tabular[1:]), tf.cast(tabular[0], tf.float32)

def preprocess_validation(image_url, tabular):
    image_string = tf.io.read_file(image_url)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.image.central_crop(image, 1.0)
    image = tf.image.resize(image, (image_size, image_size))
    return (image, tabular[1:]), tf.cast(tabular[0], tf.float32)

In [7]:
def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean((y_true -  y_pred) ** 2))

In [8]:
# def base_model():
#     efficient_net = tf.keras.applications.EfficientNetB0(
#         weights = "../input/keras-applications-models/EfficientNetB0.h5", 
#         include_top = False, 
#         input_shape = (image_size, image_size, 3)
#     )    
#     efficient_net.trainable = False
#     return efficient_net

In [9]:
# efficient_net = base_model()

In [10]:
from keras.applications import vgg16

In [11]:
def base_model():
    resnet50_net = tf.keras.applications.resnet50.ResNet50(
        weights = "../input/keras-pretrained-models/ResNet50_NoTop_ImageNet.h5", 
        include_top = False, 
        input_shape = (image_size, image_size, 3)
    )    
    resnet50_net.trainable = False
    return resnet50_net

In [12]:
resnet50_net = base_model()

2021-12-31 09:29:57.939204: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-31 09:29:58.044668: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-31 09:29:58.045364: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-31 09:29:58.046942: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [13]:
def get_tabular_model(inputs):
#     width = 32
#     depth = 3
#     activation = "relu"
#     kernel_regularizer = keras.regularizers.l2()
#     x = keras.layers.Dense(width, activation=activation, kernel_regularizer=kernel_regularizer)(inputs)
#     for i in range(depth):
#         if i == 0:
#             x = inputs
#         x = keras.layers.Dense(
#             width, 
#             activation=activation,
#             kernel_regularizer=kernel_regularizer
#         )(x)
#         if (i + 1) % 3 == 0:
#             x = keras.layers.Concatenate()([x, inputs])
    activation = "relu"
    kernel_regularizer = keras.regularizers.l2()
    x = keras.layers.Dense(12, activation=activation, kernel_regularizer = keras.regularizers.l2())(inputs)
    x = keras.layers.Dense(64, activation=activation,kernel_regularizer = keras.regularizers.l2())(x)
    x = keras.layers.Dropout(0.3)(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dense(128, activation=activation,kernel_regularizer = keras.regularizers.l2())(x)
    x = keras.layers.Dense(64, activation=activation,kernel_regularizer = keras.regularizers.l2())(x)
    x = keras.layers.Concatenate()([x, inputs])
    return x

In [14]:
def get_model():
    image_inputs = tf.keras.Input((image_size, image_size , 3))
    tabular_inputs = tf.keras.Input(len(tabular_columns))
    image_x = resnet50_net(image_inputs)
    image_x = tf.keras.layers.GlobalAveragePooling2D()(image_x)
    #for _ in range(1):
    #    image_x = tf.keras.layers.Dense(256, kernel_regularizer=tf.keras.regularizers.l2())(image_x)
    image_x = tf.keras.layers.Dropout(0.5)(image_x)
    tabular_x = get_tabular_model(tabular_inputs)
    x = tf.keras.layers.Concatenate(axis=1)([image_x, tabular_x])
    output = tf.keras.layers.Dense(1)(x)
    model = tf.keras.Model(inputs=[image_inputs, tabular_inputs], outputs=[output])
    return model

In [15]:
show_architecture = True
if show_architecture:
    model = get_model()
    tf.keras.utils.plot_model(model, show_shapes=True)
    print(model.summary())
    image = np.random.normal(size=(1, image_size, image_size, 3))
    tabular = np.random.normal(size=(1, len(tabular_columns)))
    print(image.shape, tabular.shape)
    print(model((image, tabular)).shape)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 12)]         0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 12)           156         input_3[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 64)           832         dense[0][0]                      
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 64)           0           dense_1[0][0]                    
______________________________________________________________________________________________

2021-12-31 09:30:06.734331: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


(1, 1)


In [16]:
tf.keras.backend.clear_session()
models = []
histories = []
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
for index, (train_indices, val_indices) in enumerate(kfold.split(train)):
    if train_on_fold != None and train_on_fold != index:
        continue
    x_train = train.loc[train_indices, "file_path"]
    tabular_train = train.loc[train_indices, ["Pawpularity"] + tabular_columns]
    x_val= train.loc[val_indices, "file_path"]
    tabular_val = train.loc[val_indices, ["Pawpularity"] + tabular_columns]
    checkpoint_path = "model_%d.h5"%(index)
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        checkpoint_path, 
        save_best_only=True
    )
    early_stop = tf.keras.callbacks.EarlyStopping(
        min_delta=1e-4, 
        patience=10
    )
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
        factor=0.3,
        patience=2, 
        min_lr=1e-7
    )
    
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-3,
        decay_steps=1000,
        decay_rate=0.9,
        staircase=True)
    
    callbacks = [early_stop, checkpoint, reduce_lr]
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
    
    train_ds = tf.data.Dataset.from_tensor_slices((x_train, tabular_train)).map(preprocess_training).shuffle(512).batch(batch_size).cache().prefetch(AUTO)
    val_ds = tf.data.Dataset.from_tensor_slices((x_val, tabular_val)).map(preprocess_validation).batch(batch_size).cache().prefetch(AUTO)
    model = get_model()
    model.compile(loss=rmse, optimizer=optimizer, metrics=["mae", "mape"])
    history = model.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks=callbacks)
#     for metrics in [("loss", "val_loss"), ("mae", "val_mae"), ("mape", "val_mape"), ["lr"]]:
#         pd.DataFrame(history.history, columns=metrics).plot()
#         plt.show()
    model.load_weights(checkpoint_path)
    histories.append(history)
    models.append(model)

Epoch 1/20


2021-12-31 09:30:14.845871: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-31 09:30:25.368733: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 498 of 512
2021-12-31 09:30:25.668654: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:228] Shuffle buffer filled.


124/124 [==============================] - 174s 1s/step - loss: 24.8052 - mae: 18.0316 - mape: 75.9236 - val_loss: 22.3742 - val_mae: 15.6482 - val_mape: 71.2250


/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/20
124/124 [==============================] - 16s 128ms/step - loss: 21.0335 - mae: 15.2779 - mape: 76.6189 - val_loss: 21.3248 - val_mae: 15.1135 - val_mape: 70.1250
Epoch 3/20
124/124 [==============================] - 16s 126ms/step - loss: 20.3851 - mae: 14.9513 - mape: 76.0100 - val_loss: 20.6513 - val_mae: 14.8574 - val_mape: 71.5228
Epoch 4/20
124/124 [==============================] - 16s 128ms/step - loss: 20.0376 - mae: 14.7350 - mape: 75.9898 - val_loss: 20.2421 - val_mae: 14.7589 - val_mape: 72.8428
Epoch 5/20
124/124 [==============================] - 16s 126ms/step - loss: 19.7687 - mae: 14.6089 - mape: 75.6443 - val_loss: 20.0302 - val_mae: 14.6579 - val_mape: 73.0459
Epoch 6/20
124/124 [==============================] - 16s 127ms/step - loss: 19.6462 - mae: 14.5333 - mape: 75.1109 - val_loss: 19.8850 - val_mae: 14.6401 - val_mape: 73.7070
Epoch 7/20
124/124 [==============================] - 16s 127ms/step - loss: 19.5692 - mae: 14.4834 - mape: 75.1640 - val_los

...

Epoch 2/20
124/124 [==============================] - 16s 127ms/step - loss: 21.3122 - mae: 15.4271 - mape: 76.8526 - val_loss: 20.4509 - val_mae: 14.4834 - val_mape: 72.5602
Epoch 3/20
124/124 [==============================] - 16s 127ms/step - loss: 20.5686 - mae: 15.0188 - mape: 75.8339 - val_loss: 19.9835 - val_mae: 14.1688 - val_mape: 70.7515
Epoch 4/20
124/124 [==============================] - 16s 126ms/step - loss: 20.2717 - mae: 14.8438 - mape: 75.0890 - val_loss: 19.5694 - val_mae: 14.0643 - val_mape: 72.4949
Epoch 5/20
124/124 [==============================] - 16s 126ms/step - loss: 20.0518 - mae: 14.7329 - mape: 74.9849 - val_loss: 19.3652 - val_mae: 13.9843 - val_mape: 72.5817
Epoch 6/20
124/124 [==============================] - 16s 126ms/step - loss: 19.8450 - mae: 14.5954 - mape: 74.5918 - val_loss: 19.2435 - val_mae: 13.9503 - val_mape: 73.0121
Epoch 7/20
124/124 [==============================] - 16s 126ms/step - loss: 19.7750 - mae: 14.5904 - mape: 74.3549 - val_los

/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/20
124/124 [==============================] - 16s 127ms/step - loss: 21.3378 - mae: 15.4904 - mape: 77.1170 - val_loss: 20.5630 - val_mae: 14.6959 - val_mape: 72.8419
Epoch 3/20
124/124 [==============================] - 16s 125ms/step - loss: 20.6786 - mae: 15.0971 - mape: 76.0918 - val_loss: 20.0197 - val_mae: 14.4099 - val_mape: 72.8618
Epoch 4/20
124/124 [==============================] - 16s 126ms/step - loss: 20.2657 - mae: 14.8610 - mape: 75.7288 - val_loss: 19.6924 - val_mae: 14.2774 - val_mape: 73.3195
Epoch 5/20
124/124 [==============================] - 16s 128ms/step - loss: 20.1028 - mae: 14.7655 - mape: 75.2426 - val_loss: 19.5232 - val_mae: 14.1418 - val_mape: 72.6781
Epoch 6/20
124/124 [==============================] - 16s 126ms/step - loss: 19.8614 - mae: 14.5937 - mape: 75.2158 - val_loss: 19.3644 - val_mae: 14.1823 - val_mape: 74.7243
Epoch 7/20
124/124 [==============================] - 16s 127ms/step - loss: 19.7368 - mae: 14.5314 - mape: 74.7393 - val_los

/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/20
124/124 [==============================] - 16s 127ms/step - loss: 21.1533 - mae: 15.3855 - mape: 77.3835 - val_loss: 21.1715 - val_mae: 14.7892 - val_mape: 67.8705
Epoch 3/20
124/124 [==============================] - 16s 126ms/step - loss: 20.4025 - mae: 14.9291 - mape: 76.6099 - val_loss: 20.6754 - val_mae: 14.5236 - val_mape: 68.0187
Epoch 4/20
124/124 [==============================] - 16s 127ms/step - loss: 20.1073 - mae: 14.7389 - mape: 75.7556 - val_loss: 20.2566 - val_mae: 14.4261 - val_mape: 70.2789
Epoch 5/20
124/124 [==============================] - 16s 126ms/step - loss: 19.7743 - mae: 14.5468 - mape: 75.3568 - val_loss: 19.9543 - val_mae: 14.5274 - val_mape: 74.4387
Epoch 6/20
124/124 [==============================] - 16s 127ms/step - loss: 19.6458 - mae: 14.5083 - mape: 74.9817 - val_loss: 19.8821 - val_mae: 14.4640 - val_mape: 74.0668
Epoch 7/20
124/124 [==============================] - 16s 126ms/step - loss: 19.4579 - mae: 14.3694 - mape: 74.3763 - val_los

In [17]:
def preprocess_test_data(image_url, tabular):
    print(image_url, tabular)
    image_string = tf.io.read_file(image_url)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.image.central_crop(image, 1.0)
    image = tf.image.resize(image, (image_size, image_size))
    # 0 won't be used in prediction, but it's needed in this senario or the tabular variable is treated as label.
    return (image, tabular), 0

In [18]:
test_ds = tf.data.Dataset.from_tensor_slices((test["file_path"], test[tabular_columns])).map(preprocess_test_data).batch(batch_size).cache().prefetch(2)

Tensor("args_0:0", shape=(), dtype=string) Tensor("args_1:0", shape=(12,), dtype=int64)


In [19]:
total_results = []
for model in models:
    total_results.append(model.predict(test_ds).reshape(-1))
results = np.mean(total_results, axis=0).reshape(-1)

In [20]:
sample_submission["Pawpularity"] = results
sample_submission.to_csv("submission.csv", index=False)
sample_submission

,Id,Pawpularity
0,4128bae22183829d2b5fea10effdb0c3,36.005703
1,43a2262d7738e3d420d453815151079e,31.368555
2,4e429cead1848a298432a0acad014c9d,32.342007
3,80bc3ccafcc51b66303c2c263aa38486,29.542236
4,8f49844c382931444e68dffbe20228f4,26.014105
5,b03f7041962238a7c9d6537e22f9b017,34.133095
6,c978013571258ed6d4637f6e8cc9d6a3,25.989162
7,e0de453c1bffc20c22b072b34b54e50f,30.432556
